In [1]:
import pandas
from os.path import isfile, join
import os


root_path = os.getcwd()

def limpaDf(df):
        new = df[ ["Ação Orçamentária", "Município", "Valor Transferido"]].copy() 
        return new

def limpaDfIDEB(df):
        return df[ ["Municipio", 2017]]
        
        

In [5]:
path = "./dados/dados brutos/Transferencias 2016"
directory = os.listdir(path)
all_data_frames = []

for file in directory:
    file_path = os.path.join(path, file)
    df = pandas.read_csv( file_path ,  delimiter=";", encoding="utf-8-sig")
    df = limpaDf(df)
    all_data_frames.append(df)

acoes2016 = pandas.concat(all_data_frames)
acoes2016.to_csv(r"./dados/dados limpos/acoes2016.csv")

In [3]:
collection_data = []
columns_collection = ["Município", "Total Investido", "IDEB - 4 e 5", "IDEB - 8 e 9"]

df_ideb_5_6 = pandas.read_excel("./dados/dados brutos/IDEB Paraíba.xlsx", sheet_name="4 serie5 ano",  delimiter=";", encoding="utf-8-sig")
df_ideb_8_9 = pandas.read_excel("./dados/dados brutos/IDEB Paraíba.xlsx", sheet_name="8 Serie 9 ano",  delimiter=";", encoding="utf-8-sig")

df_ideb_5_6 = limpaDfIDEB(df_ideb_5_6)
df_ideb_8_9 = limpaDfIDEB(df_ideb_8_9)


In [7]:
import pandasql as ps



file_path =  "./dados/dados limpos/acoes2016.csv"
file_path2017 =  "./dados/dados limpos/acoes2017.csv"
acoes = pandas.read_csv( file_path ,  delimiter=",", encoding="utf-8-sig")
acoes2017 = pandas.read_csv( file_path2017 ,  delimiter=",", encoding="utf-8-sig")

acoes = pandas.concat([acoes,acoes2017])

municipios = acoes[["Município"]].copy()

municipios = municipios.drop_duplicates(subset="Município")

municipios_list = list(municipios["Município"])

for municipio in municipios_list :
    df_municipio = ps.sqldf('select * from acoes where Município = "' + municipio + '"', locals())
    ideb_municipio_4_5 = ps.sqldf('select \"2017\" from df_ideb_5_6 where '\
                              'Municipio like "' + municipio.capitalize() + '"', locals())
    ideb_municipio_8_9 = ps.sqldf('select \"2017\" from df_ideb_8_9 where '\
                              'Municipio like "' + municipio.capitalize() + '"', locals())
    total = 0
    
    for row in df_municipio["Valor Transferido"]:
        row = row.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
        total += float(row)
    
    collection_data.append([
        municipio,
        total,
        "" if len(ideb_municipio_4_5["2017"]) == 0 else ideb_municipio_4_5["2017"][0],
        "" if len(ideb_municipio_8_9["2017"]) == 0 else ideb_municipio_8_9["2017"][0],
    ])
    
    df_municipio.to_csv(r"./dados/dados limpos/acoes-municipio/" + municipio + ".csv")

pandas.DataFrame(
    collection_data, columns=columns_collection
).to_csv(r"./dados/dados limpos/ideb-municipio-por-investimento.csv")

In [8]:
file_path = "./dados/dados brutos/matriculas2017.csv"
matriculas = pandas.read_csv( file_path ,  delimiter=",", encoding="utf-8-sig")


In [9]:
from IPython.display import display, HTML

teste = ps.sqldf('select * from matriculas where "Unidades" != "Estadual Urbana" ', locals())
teste = ps.sqldf('select * from teste where "Unidades" != "Estadual Rural" ', locals())
teste = ps.sqldf('select * from teste where "Unidades" != "Estadual e Municipal" ', locals())



In [10]:

for index, row in teste.iterrows():
    if(row['Unidades'] != "Municipal Urbana" and row['Unidades'] != "Municipal Rural" ):
        teste['Total'][index] = teste['Total'][index+1] + teste['Total'][index+2]
                
display(teste)

teste = ps.sqldf('select * from teste where "Unidades" != "Municipal Urbana" ', locals())
teste = ps.sqldf('select * from teste where "Unidades" != "Municipal Rural" ', locals())

matriculados_municipio = teste[ ["Unidades", "Total"]]
display(matriculados_municipio)

matriculados_municipio.to_csv(r"./dados/dados limpos/matriculados_municipal_2017.csv")


C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Unidades,Parcial AI,Integral AI,Parcial AF,Integral AF,Total
0,PARAIBA,NaN,NaN,NaN,NaN,329842
1,Municipal Urbana,92566.0,43465.0,82937.0,26261.0,245229
2,Municipal Rural,44153.0,21129.0,11730.0,7601.0,84613
3,AGUA BRANCA,NaN,NaN,NaN,NaN,1523
4,Municipal Urbana,389.0,114.0,374.0,115.0,992
5,Municipal Rural,334.0,53.0,93.0,51.0,531
6,AGUIAR,NaN,NaN,NaN,NaN,593
7,Municipal Urbana,256.0,0.0,226.0,0.0,482
8,Municipal Rural,111.0,0.0,0.0,0.0,111
9,ALAGOA GRANDE,NaN,NaN,NaN,NaN,2826


,Unidades,Total
0,PARAIBA,329842
1,AGUA BRANCA,1523
2,AGUIAR,593
3,ALAGOA GRANDE,2826
4,ALAGOA NOVA,1848
5,ALAGOINHA,1769
6,ALCANTIL,727
7,ALGODAO DE JANDAIRA,521
8,ALHANDRA,2290
9,AMPARO,396


In [11]:
file_path = "./dados/dados limpos/ideb-municipio-por-investimento.csv"


df_municipio_invest = pandas.read_csv( file_path ,  delimiter=",", encoding="utf-8-sig")

df_municipio_invest["Matriculas"] = ''

df_municipio_invest["Investimento por matricula"] = ''

file_path = "./dados/dados limpos/matriculados_municipal_2017.csv"
matriculados_municipio =  pandas.read_csv( file_path ,  delimiter=",", encoding="utf-8-sig") 

In [12]:
for index, row in df_municipio_invest.iterrows():

        matricula_mun = ps.sqldf('select * from matriculados_municipio where "Unidades" = "' + row['Município'] + '"'  , locals())
        try:
            df_municipio_invest['Matriculas'][index] = matricula_mun.at[0,'Total']
            df_municipio_invest['Investimento por matricula'][index] =  df_municipio_invest['Total Investido'][index]/df_municipio_invest['Matriculas'][index]
        except:
            df_municipio_invest['Matriculas'][index] = 0
            df_municipio_invest['Investimento por matricula'][index] = 0

C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

...

C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be se

C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aaron\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [13]:
df_municipio_invest.to_csv(r"./dados/dados limpos/ideb_investimento_relativo.csv")